<a href="https://colab.research.google.com/github/KEkerete/FadePredictionUsingTensoflow/blob/main/SwitchingwGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 20 01:15:39 2023

@author: kdellubuntu
"""

# -*- coding: utf-8 -*-
"""
Created on Thu Jun 27 20:20:04 2019

@author: ke0015
"""

import sys
sys.modules[__name__].__dict__.clear()

#%reset -f
import time
start_time = time.time()

import dill
import csv
import numpy as np
#import pandas as pd
from sys import platform
from pandas import read_csv
import GRU_Stacked2 as pr  #RandForest as pr
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

def debug_signal_handler(signal, frame):  #pause execution with ctl+c, resume with c
    import pdb
    pdb.set_trace()
import signal
signal.signal(signal.SIGINT, debug_signal_handler)

#%% -init-
maxMCNdB = 19.57        # maximum for ModCod
totalSpecEff = 0

trainingPctge = 0.8     # Training percentage

specEffData = []  #siteIdxNFade, , []
wSwitchData = tempSwitchData = [[0, 0, 0.0, 0, '00000000-00000', 0]]

predictAheadMins = 5
minSwitchingThreshold, maxSwitchingThreshold = 12, 14 # dB
currentSiteCol = 1
currentFadeLevelCol = 2

#%% load dataset
# series = read_csv('/home/kdellubuntu/Python/MayJul17Cured.csv')
series = read_csv('/home/kdellubuntu/Python/researchData/ChilboltonChiltonFadeData.csv')
ModCod = read_csv('/home/kdellubuntu/Python/researchData/ModCod.csv')

# csv_file_data = pd.read_csv('/home/kdellubuntu/Python/MetNFadeDataChilbolton.csv')
# # csv_filename = pd.read_csv('/home/kdellubuntu/Python/TestDataChilEdin.csv')
# input_feature = csv_file_data[['y']]

totalAttenData = series.values

#%%  Correlation between data
# fadeData = series[['Chilbolton','Chilton']]
# correlation = fadeData.corr(method='pearson')
# print(correlation)
# plt.scatter(series.loc[:, 'Chilbolton'], series.loc[:, 'Chilton'])
# plt.show()

#%% Switching
if totalAttenData[0][1] < totalAttenData[0][2]:  # Initial starting site
    rxSite = 1  # Chilbolton
    altSite = 2
else:
    rxSite = 2  # Chilton
    altSite = 1

predictAheadData = predictAheadMins * 60    # seconds
trainData = int(predictAheadData * trainingPctge / (1 - trainingPctge))

#%% Loop for dB  minSwitchingThreshold, switchingThreshold
for switchingThreshold in range(minSwitchingThreshold, maxSwitchingThreshold, 2):
    sameSiteCount, totalSpecEff = 0, 0

    zeroCol = np.zeros((len(totalAttenData), 1))
    tData = np.hstack((totalAttenData, zeroCol, zeroCol))
    totalAttenData = tData
    del tData

    currentSiteCol += 2
    currentFadeLevelCol += 2

    for k in range(0, len(totalAttenData)):
        snrdB = maxMCNdB - totalAttenData[k][rxSite] + 0.5      # snr
        specEffIdx = ModCod[(ModCod['SNRdB-upper'] >= snrdB) & (ModCod['SNRdB-lower'] < snrdB)].values
        specEff = specEffIdx[0][4]
        totalSpecEff += specEff             # total spectral efficiency
        sameSiteCount += 1

        if k > trainData:
            if totalAttenData[k][rxSite] > switchingThreshold:
                siteData = totalAttenData[k - trainData : k + predictAheadData,:]
                siteData = siteData[:,rxSite]

                #save workspace
                dill.dump_session('globalsave.pkl')

                # quit() ############

                # YPred, cErr = pr.RandForest(siteData)
                # YPred, trngErr, valErr = pr.GRU_Stacked(siteData)
                YPred = pr.GRU_Stacked2(siteData)
                if np.amax(YPred) > totalAttenData[k][rxSite]:   # check if max(predictedFade) > currentFadeLevel

                    # print("rxSite : {:6.3f}".format(rxSite))
                    altSite = (rxSite % 2) + 1      # Alternate site
                    # print("altSite : {:6.3f}".format(altSite))
                    # input("Press any key!")

                if totalAttenData[k][altSite] < totalAttenData[k][rxSite]:   # check if fade(otherSite) < fade(currentSite)
                    siteData = totalAttenData[k - trainData : k + predictAheadData,:]
                    siteData = siteData[:,rxSite]

                    #save workspace
                    dill.dump_session('globalsave.pkl')

                    # YPred, cErr = pr.RandForest(siteData)
                    # YPred, trngErr, valErr = pr.GRU_Stacked(siteData)
                    YPred = pr.GRU_Stacked2(siteData)
                    if np.amax(YPred) > totalAttenData[k][rxSite]:    # check if max(predictedFade) > currentFadeLevel in the alt site
                        if sameSiteCount >= 60: # must sustain for more than a minute
                            tempSwitchData.append([switchingThreshold, rxSite, totalAttenData[k-1][rxSite], sameSiteCount, totalAttenData[k][0], k])
                            wSwitchData = np.vstack((wSwitchData, tempSwitchData))
                            rxSite = altSite   # Switch to the alternate site

                        sameSiteCount = 0  # reset
                        tempSwitchData = [[0, 0, 0.0, 0, '00000000-00000', 0]]

        rxFade = totalAttenData[k][rxSite]
        totalAttenData[k, currentSiteCol] = rxSite
        totalAttenData[k, currentFadeLevelCol] = rxFade


        if k % 10000 == 0:
            if rxSite == 1:
                siteLoc = "Chilbolton"
            else:
                siteLoc = "Chilton"

            # print("\nDateTime={}, rxSite={}, Thresh(dB)={:,}/{:,} [{:,}/{:,}]\n".format(totalAttenData[k][0].strip(), siteLoc, switchingThreshold, maxSwitchingThreshold, k, len(totalAttenData)))
            print("\nTime={}, rxSite={}, Thresh(dB)={:,}/{:,} [{:,}/{:,}]\n".format(totalAttenData[k][0], siteLoc, switchingThreshold, maxSwitchingThreshold, k, len(totalAttenData)))


    aveSpecEff = totalSpecEff / len(totalAttenData)
    print("\n\t Average Spectral Efficiency (FER) = {:5.2f}\n".format(aveSpecEff))

    switchData = wSwitchData[wSwitchData[:, 4] != '00000000-00000']

    # input("Press any key to cintinue!")

    #%% Save results
    specEffData.append([switchingThreshold, aveSpecEff])

    if platform == 'win32':
        SwitchDataFile = "S:\Research\AlphasatData\Results\SwDataFilePred2#"+"{:,}".format(switchingThreshold)+"mins.csv"
        specEffDataFile = "S:\Research\AlphasatData\Results\SpEffDataFilePred2.csv"
    elif platform == 'linux' or platform == 'linux2':
        SwitchDataFile = "/home/kdellubuntu/Python/Results/SwDataFilePred2#"+"{:,}".format(switchingThreshold)+"mins.csv"
        specEffDataFile = "/home/kdellubuntu/Python/Results/SpEffDataFilePred2.csv"

    with open(SwitchDataFile, 'w', newline='') as outfile:
        csv.writer(outfile).writerows(switchData)

    try:
        with open(specEffDataFile, 'w', newline='') as outfile:
            csv.writer(outfile).writerow(specEffData)
    except:
        with open(specEffDataFile, 'a', newline='') as outfile:
            csv.writer(outfile).writerow(specEffData)

#%%########################
# input("Press any key to cintinue!")

color_map = plt.get_cmap('tab10')  # You can choose other color maps as well

figure(num=None, figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
plt.title("Rain Fades and Switching for Chilbolton and Chilton", fontsize=14)  #%%s current_date));
# plt.plot([row[1] for row in totalAttenData], "b", markersize=10, label="rxFade")  #
# plt.plot([row[0] for row in totalAttenData], "r", markersize=10, label="Site (1 = Chilbolton, 2 = Edinburgh)",linewidth=2.0)  # rxSite
plt.plot(totalAttenData[:,1], "r", markersize=10, label="Chilbolton")  #
plt.plot(totalAttenData[:,2], "b", markersize=10, label="Chilton")  #

j = 3
for idx in range(minSwitchingThreshold, maxSwitchingThreshold, 2 ):
    color = color_map(idx)
    plt.plot(2*j + totalAttenData[:,j], color=color, markersize=10, label=f'{idx} dB')  #
    j=j+2

plt.xlim(0, 2000000)  # len(totalAttenData))
plt.legend(loc="best")
plt.xlabel("Time (seconds)")
plt.ylabel("Rain fade (dB)")
plt.grid(which='major')
plt.show()

elapsed_time = time.time() - start_time   #toc
print("elapsedTime : {:9.9f} s".format(elapsed_time))
print('\nFin!\n')


#%%
def GRU_Stacked2(data):

    # near perfect!
    import numpy as np
    import pandas as pd
    # import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    from sklearn.metrics import mean_squared_error
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

    # Load dataset
    # data = pd.read_csv('/home/kdellubuntu/researchData/TestDataChilEdin.csv')
    df = pd.DataFrame(data)   #, index=False, header=False)   # df = data

    # Experiment with different window sizes
    window_sizes = [5, 10, 15]

    for window_size in window_sizes:
        # Scaling the data with different scalers
        scalers = [StandardScaler(), MinMaxScaler()]

        for scaler in scalers:
            # Prepare data for training
            sequence_length = window_size
            X, y = [], []

            scaled_data = scaler.fit_transform(df)

            for i in range(len(scaled_data) - sequence_length):
                X.append(scaled_data[i:i + sequence_length])
                y.append(scaled_data[i + sequence_length])
            X, y = np.array(X), np.array(y)

            # Split data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

            # Model architecture
            model = Sequential([
                Bidirectional(GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)),
                Dropout(0.2),
                GRU(64, return_sequences=False),
                Dense(32, activation='relu'),
                Dense(1)
            ])

            model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse')

            # Early Stopping and Learning Rate Scheduler
            early_stopping = EarlyStopping(patience=15, restore_best_weights=True)
            lr_scheduler = ReduceLROnPlateau(factor=0.2, patience=5, min_lr=1e-6)

            # Train the model
            history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping, lr_scheduler])

            # Make predictions
            train_predictions = model.predict(X_train)
            test_predictions = model.predict(X_test)

            # Inverse transform predictions for visualization
            train_predictions_inv = scaler.inverse_transform(train_predictions)
            test_predictions_inv = scaler.inverse_transform(test_predictions)

            # Calculate RMSE
            train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions_inv))
            test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions_inv))

            print(f"Window Size: {window_size}, Scaler: {scaler.__class__.__name__}")
            print(f"Train RMSE: {train_rmse:.4f}")
            print(f"Test RMSE: {test_rmse:.4f}")
            print("-----------------------")

    # # Visualization
    # plt.figure(figsize=(12, 6))
    # plt.plot(df.index[sequence_length:], df['Edin'][sequence_length:], label='Original Data')
    # plt.plot(df.index[sequence_length:sequence_length + len(train_predictions_inv)], train_predictions_inv, label='Train Predictions', linestyle='dashed')
    # plt.plot(df.index[sequence_length + len(train_predictions_inv):], test_predictions_inv, label='Test Predictions', linestyle='dashed')
    # plt.xlabel('Time (secs)')
    # plt.ylabel('Fade (dB)')
    # plt.title('Time Series Prediction using Stacked GRU with Attention')
    # plt.legend()
    # plt.grid(which='major')
    # plt.show()

    return test_predictions_inv

ModuleNotFoundError: ignored